In [ ]:
import os, sys, multiprocessing
import numpy as np
home_dir = '/home/vboxuser/Projects/G4NS/G4_Nanophotonic_Scintillator/'
sys.path.insert(0, home_dir)
import analysis.read_simulation as rs
# Run the script from the build directory
import random
import numpy as np
import analysis.g2_analysis.g2 as g2a
from utils.utils import *
import matplotlib.pyplot as plt
from matplotlib import cm

def get_g2(scintillation_lifetime, rate_x_ray, nEvents):
    print("scintillation_lifetime", scintillation_lifetime, "rate_x_ray", rate_x_ray)
    allPhotonsTime = []
    time = 0
    folder_name = 'scintillation_time'
    # max_time = nEvents * 1.
    for i in range(nEvents):
        photonsTime = rs.read_simulation_field(home_dir + 'build/' + folder_name + '/' + str(scintillation_lifetime) + '/output'+str(i+1)+'.root', property='fCreatorProcess', key='Scintillation', field='fT')
        photonsTime = np.array(photonsTime) + time
        allPhotonsTime.extend(photonsTime.tolist())
        interarrival_time = random.expovariate(rate_x_ray)
        time += interarrival_time
        # if time > (max_time):
        #     break

    g20 = g2a.compute_g2(allPhotonsTime, draw=False)
    return g20

In [ ]:
# Compute the g2(0) for different x_ray rates and a given lifetime
nEvents = 3000
tau_scintillation_lifetime = 2.0
rates_x_ray = np.linspace(0.1, 0.3, 4) # Should be smaller than the rate of lifetime scintillation
def g2_at_0(rate):
    try:
        g20 = get_g2(tau_scintillation_lifetime, rate, nEvents)
    except Exception as e:
        g20 = 0
        print("error:", e)   
    return g20 

# g2s_xray_rates = parallel_for_loop(g2_at_0, rates_x_ray)
g2s_xray_rates = [g2_at_0(r) for r in rates_x_ray]

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(rates_x_ray, g2s_xray_rates)
ax.plot(rates_x_ray, 1 / rates_x_ray * rates_x_ray[0] * g2s_xray_rates[0])

ax.set_xlabel('electrons flux [1/ns]')
ax.set_ylabel('g2(0)')
ax.set_title('g2(0) as a function of the electrons flux')
plt.show()

In [ ]:
# Compute the g2(0) for different lifetimes and a given x_ray rate
nEvents = 100
taus = np.linspace(1., 10., 10)
rate_x_ray = 0.05 # Should be smaller than the rate of lifetime scintillation
def g2_at_0(tau):
    try:
        g20 = get_g2(tau, rate_x_ray, nEvents)
    except Exception as e:
        g20 = 0
        print("error:", e)   
    return g20 
g2s = parallel_for_loop(g2_at_0, taus)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(taus, g2s)
ax.plot(taus, 1/(taus) * taus[0] * g2s[0])

ax.set_xlabel('scintillation lifetime [ns]')
ax.set_ylabel('g2(0)')
ax.set_title('g2(0) as a function of the scintillation lifetime')
plt.show()

In [ ]:
# Compute the g2(0) for different scintillation yeild
nEvents = 10000
scintillationYield_array = np.logspace(2, 4, 2*multiprocessing.cpu_count() - 1)
rate_x_ray = 0.1 # Should be smaller than the rate of lifetime scintillation
def g2_at_0(scintillation_yeild):
    try:
        g20 = get_g2(scintillation_yeild, rate_x_ray, nEvents)
    except Exception as e:
        g20 = 0
        print("error:", e)   
    return g20 
g2s = parallel_for_loop(g2_at_0, scintillationYield_array)

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(scintillationYield_array / 1000 * 10, g2s)

ax.set_xlabel('scintillation yeild [light photons / electron]')
ax.set_ylabel('g2(0)')
ax.set_title('g2(0) as a function of the scintillation yeild')
plt.show()